# Create Feature_Usage Objects in ADLSTE_COA 

In [94]:
%connect name=Transcend

Success: 'Transcend' connection established


<br/><br/><br/>
## Table - PERM

In [ ]:
CREATE MULTISET TABLE adlste_coa.coat_dat_feature_usage_log 
(
  Site_Id VARCHAR(128) CHARACTER SET LATIN  NOT NULL
, LogDate DATE FORMAT 'yyyy-mm-dd' NOT NULL
, Feature_Id INTEGER NOT NULL
, DBS_Version VARCHAR(16) CHARACTER SET LATIN COMPRESS('DEFAULT','ALT01','ALT02')
, User_Bucket VARCHAR(128) CHARACTER SET LATIN 
, User_Department VARCHAR(128) CHARACTER SET LATIN
, User_Subdepartment VARCHAR(128) CHARACTER SET LATIN
, User_Region VARCHAR(128) CHARACTER SET LATIN
, Feature_Usage_Cnt INTEGER
, Run_Id INTEGER
) PRIMARY INDEX ( Site_Id ,LogDate ,Feature_Id ) 
PARTITION BY RANGE_N(LogDate BETWEEN DATE '2019-01-01' AND DATE '2021-12-31' EACH INTERVAL '1' DAY 
                     ,NO RANGE OR UNKNOWN);

In [56]:
replace view adlste_coa.coa_dat_feature_usage_log as locking row for access
select * from adlste_coa.coat_dat_feature_usage_log

Success: 10 rows affected

In [57]:
SELECT TOP 10 * FROM adlste_coa.coat_dat_feature_usage_log

Site_Id,LogDate,Feature_Id,DBS_Version,User_Bucket,User_Department,User_Subdepartment,User_Region,Feature_Usage_Cnt,Run_Id
APPLE50BLANEY,2020-05-08,24300,DEFAULT,ADHOC,,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,ADHOC,LEGAL_HOLD,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,BOBJ,OTHERS,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,CORE ETL,SYSTEM,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,ADHOC,OTHERS,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,OUTBOUND,Unknown,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,OUTBOUND,FRAUD,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,DATA_ENG,MARKETING,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,,SALESOPS,na,na,0,196
APPLE50BLANEY,2020-05-08,24300,DEFAULT,OUTBOUND,ITUNES,na,na,0,196


<br/><br/><br/>
## View:  adlste_coa.coa_dat_feature

In [344]:
replace view adlste_coa.coa_dat_feature as locking row for access
SELECT 
 coalesce(Site_ID,'') as Site_ID
,coalesce(LogDate,cast('9999-12-31' as date)) as LogDate
,Product_Category
,Feature_Category
,Feature_SubCategory
,Feature_Name
,Feature_Usage_Cnt as Usage_Cnt
,DBQL_FeatureInfo_Flag as DBQL_Flg
,MBO_Ind
,Feature_Weight as MBO_Weight
,User_Bucket
,User_Department
,Objective_Name
,Solution_Type
,Business_Value_Desc
,User_Experience_Level_Name
,Initiator_Role_Category
,Differentiator_Ind 
,Enabled_Thru
,Feature_SQL_URL
,Playbook_URL
from adlste_coa.coa_dim_feature f
left outer join adlste_coa.coa_dat_feature_usage_log d
  on d.Feature_ID = f.Feature_ID 

Success: 22 rows affected

In [345]:
select * from adlste_coa.coa_dat_feature 
where Site_ID = 'TDCloud14TD03' and LogDate = '2020-06-10' 
order by Product_Category, Feature_Category, Feature_SubCategory, Feature_Name

<br/><br/><br/>
## Table - GTT

In [15]:
-- drop table adlste_coa_stg.stg_dat_feature_usage_log;

Success: 271 rows affected

In [16]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_feature_usage_log
(
  Site_ID         VARCHAR(128) CHARACTER SET LATIN 
 ,LogDate         DATE 
 ,User_Bucket     VARCHAR(128) CHARACTER SET LATIN 
 ,User_Department VARCHAR(128) CHARACTER SET LATIN
 ,DBSVersion      CHAR(11) CHARACTER SET LATIN
 ,Request_Count   INTEGER
 ,bit016 INTEGER ,bit017 INTEGER ,bit018 INTEGER ,bit019 INTEGER 
 ,bit020 INTEGER ,bit021 INTEGER ,bit022 INTEGER ,bit023 INTEGER ,bit024 INTEGER
 ,bit025 INTEGER ,bit026 INTEGER ,bit027 INTEGER ,bit028 INTEGER ,bit029 INTEGER
 ,bit030 INTEGER ,bit031 INTEGER ,bit032 INTEGER ,bit033 INTEGER ,bit034 INTEGER
 ,bit035 INTEGER ,bit036 INTEGER ,bit037 INTEGER ,bit038 INTEGER ,bit039 INTEGER
 ,bit040 INTEGER ,bit041 INTEGER ,bit042 INTEGER ,bit043 INTEGER ,bit044 INTEGER
 ,bit045 INTEGER ,bit046 INTEGER ,bit047 INTEGER ,bit048 INTEGER ,bit049 INTEGER
 ,bit050 INTEGER ,bit051 INTEGER ,bit052 INTEGER ,bit053 INTEGER ,bit054 INTEGER
 ,bit055 INTEGER ,bit056 INTEGER ,bit057 INTEGER ,bit058 INTEGER ,bit059 INTEGER
 ,bit060 INTEGER ,bit061 INTEGER ,bit062 INTEGER ,bit063 INTEGER ,bit064 INTEGER
 ,bit065 INTEGER ,bit066 INTEGER ,bit067 INTEGER ,bit068 INTEGER ,bit069 INTEGER
 ,bit070 INTEGER ,bit071 INTEGER ,bit072 INTEGER ,bit073 INTEGER ,bit074 INTEGER
 ,bit075 INTEGER ,bit076 INTEGER ,bit077 INTEGER ,bit078 INTEGER ,bit079 INTEGER
 ,bit080 INTEGER ,bit081 INTEGER ,bit082 INTEGER ,bit083 INTEGER ,bit084 INTEGER
 ,bit085 INTEGER ,bit086 INTEGER ,bit087 INTEGER ,bit088 INTEGER ,bit089 INTEGER
 ,bit090 INTEGER ,bit091 INTEGER ,bit092 INTEGER ,bit093 INTEGER ,bit094 INTEGER
 ,bit095 INTEGER ,bit096 INTEGER ,bit097 INTEGER ,bit098 INTEGER ,bit099 INTEGER
 ,bit100 INTEGER ,bit101 INTEGER ,bit102 INTEGER ,bit103 INTEGER ,bit104 INTEGER
 ,bit105 INTEGER ,bit106 INTEGER ,bit107 INTEGER ,bit108 INTEGER ,bit109 INTEGER
 ,bit110 INTEGER ,bit111 INTEGER ,bit112 INTEGER ,bit113 INTEGER ,bit114 INTEGER
 ,bit115 INTEGER ,bit116 INTEGER ,bit117 INTEGER ,bit118 INTEGER ,bit119 INTEGER
 ,bit120 INTEGER ,bit121 INTEGER ,bit122 INTEGER ,bit123 INTEGER ,bit124 INTEGER
 ,bit125 INTEGER ,bit126 INTEGER ,bit127 INTEGER ,bit128 INTEGER ,bit129 INTEGER
 ,bit130 INTEGER ,bit131 INTEGER ,bit132 INTEGER ,bit133 INTEGER ,bit134 INTEGER
 ,bit135 INTEGER ,bit136 INTEGER ,bit137 INTEGER
 ) NO PRIMARY INDEX 
   ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

<br/><br/><br/>
## Stored Proc

In [65]:
REPLACE PROCEDURE adlste_coa.sp_dat_feature_usage_log ( spversion VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 

    /* start setup audit process */
    DECLARE runid  INTEGER;
    DECLARE siteid VARCHAR(128);
    DECLARE tablename VARCHAR(128);
    DECLARE callingsp VARCHAR(128); 
    DECLARE startdate DATE;
    DECLARE enddate DATE;
    
    /* for reconcile: */
    DECLARE dim INTEGER; -- dim_feature
    DECLARE gtt INTEGER; -- source GTT
    DECLARE unp INTEGER; -- unpivot
    DECLARE dat INTEGER; -- final dat_feature_usage_log
    
    DECLARE err BYTEINT; -- Flag if error found
    DECLARE msg VARCHAR(100);
    
    SET runid = 0;
    SET tablename = 'adlste_coa.coat_dat_feature_usage_log';
    SET callingsp = 'adlste_coa.sp_dat_feature_usage_log';
    SET siteid = 'Empty Table';
    SET startdate = DATE;
    SET enddate = DATE;

    SELECT 
     coalesce(Site_ID,'empty table')       
    ,min(LogDate) as StartDate
    ,max(LogDate) as EndDate
    INTO siteid, startdate, enddate
    FROM adlste_coa_stg.stg_dat_feature_usage_log
    Group by 1
    ;
    
    /* end setup audit process */
    
    
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'START',  '') ;
                              
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'STEP',  'Unpivot from source') ;
        
    /* unpivot data from source */
    create volatile table feature_usage_unpivot as
    (
        select *
        from adlste_coa_stg.stg_dat_feature_usage_log
        unpivot(Usage_Cnt for BitPos in
            (bit016 as '016' ,bit017 as '017' ,bit018 as '018' ,bit019 as '019' 
            ,bit020 as '020' ,bit021 as '021' ,bit022 as '022' ,bit023 as '023' ,bit024 as '024'
            ,bit025 as '025' ,bit026 as '026' ,bit027 as '027' ,bit028 as '028' ,bit029 as '029'
            ,bit030 as '030' ,bit031 as '031' ,bit032 as '032' ,bit033 as '033' ,bit034 as '034'
            ,bit035 as '035' ,bit036 as '036' ,bit037 as '037' ,bit038 as '038' ,bit039 as '039'
            ,bit040 as '040' ,bit041 as '041' ,bit042 as '042' ,bit043 as '043' ,bit044 as '044'
            ,bit045 as '045' ,bit046 as '046' ,bit047 as '047' ,bit048 as '048' ,bit049 as '049'
            ,bit050 as '050' ,bit051 as '051' ,bit052 as '052' ,bit053 as '053' ,bit054 as '054'
            ,bit055 as '055' ,bit056 as '056' ,bit057 as '057' ,bit058 as '058' ,bit059 as '059'
            ,bit060 as '060' ,bit061 as '061' ,bit062 as '062' ,bit063 as '063' ,bit064 as '064'
            ,bit065 as '065' ,bit066 as '066' ,bit067 as '067' ,bit068 as '068' ,bit069 as '069'
            ,bit070 as '070' ,bit071 as '071' ,bit072 as '072' ,bit073 as '073' ,bit074 as '074'
            ,bit075 as '075' ,bit076 as '076' ,bit077 as '077' ,bit078 as '078' ,bit079 as '079'
            ,bit080 as '080' ,bit081 as '081' ,bit082 as '082' ,bit083 as '083' ,bit084 as '084'
            ,bit085 as '085' ,bit086 as '086' ,bit087 as '087' ,bit088 as '088' ,bit089 as '089'
            ,bit090 as '090' ,bit091 as '091' ,bit092 as '092' ,bit093 as '093' ,bit094 as '094'
            ,bit095 as '095' ,bit096 as '096' ,bit097 as '097' ,bit098 as '098' ,bit099 as '099'
            ,bit100 as '100' ,bit101 as '101' ,bit102 as '102' ,bit103 as '103' ,bit104 as '104'
            ,bit105 as '105' ,bit106 as '106' ,bit107 as '107' ,bit108 as '108' ,bit109 as '109'
            ,bit110 as '110' ,bit111 as '111' ,bit112 as '112' ,bit113 as '113' ,bit114 as '114'
            ,bit115 as '115' ,bit116 as '116' ,bit117 as '117' ,bit118 as '118' ,bit119 as '119'
            ,bit120 as '120' ,bit121 as '121' ,bit122 as '122' ,bit123 as '123' ,bit124 as '124'
            ,bit125 as '125' ,bit126 as '126' ,bit127 as '127' ,bit128 as '128' ,bit129 as '129'
            ,bit130 as '130' ,bit131 as '131' ,bit132 as '132' ,bit133 as '133' ,bit134 as '134'
            ,bit135 as '135' ,bit136 as '136' ,bit137 as '137' ) 
        ) tmp
    ) with data 
      no primary index
      on commit preserve rows;
      

    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'STEP',  'Delete from Target') ;
    
    DELETE from adlste_coa.coat_dat_feature_usage_log 
    where (Site_ID, LogDate) in (select Site_ID, LogDate from feature_usage_unpivot);

                              
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'STEP',  'Join to DIM_Feature and insert into Target') ;

    INSERT into adlste_coa.coat_dat_feature_usage_log 
    select u.Site_ID, u.LogDate, f.Feature_ID
    ,case when u.DBSVersion between '16.20.33.01' and '16.20.53.07' then 'ALT01'
          else 'DEFAULT' end as DBS_Version
    ,u.User_Bucket, u.User_Department, 'na' as User_SubDepartment, 'na' as User_Region
    ,Usage_Cnt as Feature_Usage_Cnt
    ,:runid as RunID
    from feature_usage_unpivot u
    join adlste_coa.coat_dim_feature f
      on cast(u.BitPos as int) = f.Feature_BitPos
     and f.DBS_Version = case 
                         when u.DBSVersion between '16.20.33.01' and '16.20.53.07' then 'ALT01'
                         else 'DEFAULT' end
    ;
      


    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'STEP',  'Reconcile record counts');
    

    /* Reconcile */  
    Select gtt, unp, dim, dat 
      INTO gtt, unp, dim, dat
    from 
    (select count(*) as gtt from dbc.columns
     where databasename = 'adlste_coa_stg'
       and tablename = 'stg_dat_feature_usage_log'
       and columnname like 'bit%') g
    cross join
    (select count(distinct bitpos) as unp from feature_usage_unpivot) u
    cross join
    (select count(*) as dim from adlste_coa.coat_dim_feature where feature_bitpos <>0) i
    cross join
    (select count(distinct feature_ID) as dat from adlste_coa.coat_dat_feature_usage_log
     where Site_ID= :siteid and LogDate between :startdate AND :enddate ) a
     ;
     
     
    IF gtt <> unp THEN
        CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                          'normal run', 'ERROR',  'Global Temp Table different than Unpivot Table') ;
    END IF;
                          
    IF unp <> dim THEN        
        Select 'MISSING BIT POSITION DEFINITION: ' || min(u.BitPos) || ' -- found in '|| trim(u.Site_ID) ||' ('|| u.DBSVersion ||')' as msg
        INTO msg
        from feature_usage_unpivot u
        left outer join adlste_coa.coat_dim_feature f
          on cast(u.BitPos as int) = f.Feature_BitPos
        where f.Feature_ID is null
        group by u.Site_ID, u.DBSVersion;
        
        CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                      'normal run', 'ERROR',  :msg) ;
    END IF;
        
        
    IF dim <> dat THEN
        CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                          'normal run', 'ERROR',  'DIM_Feature different than DAT_Feature_Usage_Log') ;
    END IF;


    drop table feature_usage_unpivot;
    delete from adlste_coa_stg.stg_dat_feature_usage_log ;

        
    CALL adlste_coa.sp_audit_log(runid, :siteid, :tablename, :callingsp, :spversion, :startdate, :enddate,
                              'normal run', 'END', '') ;

END;

In [66]:
call adlste_coa.sp_dat_feature_usage_log ('v1')

Success: 0 rows affected

<br/><br/><br/>
## Feature_Usage_LOG  sql for COA

<br><br><br>
### Customer System SQL

In [259]:
/*  extracts the feature logging from dbqlogtbl WITHOUT the cartesian Join
    by user bucket / department. Mapping to Feature_IDs happen in Transcend. 
    
  parameters
     dbqlogtbl  = {dbqlogtbl}
     siteid     = {siteid}
     startdate  = {startdate}
     enddate    = {enddate}
*/


/*{{save:feature_department.csv}}*/
/*{{load:{db_stg}.stg_dat_feature_usage_log}}*/
/*{{call:{db_coa}.sp_dat_feature_usage_log}}*/
SELECT
 '{siteid}' (VARCHAR(100)) as SiteID
,A.LogDate as LogDate
,u.User_Bucket
,u.User_Department
,(Select trim(infoData) as DBSVersion from dbc.dbcinfo where InfoKey = 'VERSION') AS DBSVersion
,count(*) as Request_Count
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -016)))) AS bit016
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -017)))) AS bit017
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -018)))) AS bit018
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -019)))) AS bit019
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -020)))) AS bit020
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -021)))) AS bit021
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -022)))) AS bit022
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -023)))) AS bit023
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -024)))) AS bit024
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -025)))) AS bit025
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -026)))) AS bit026
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -027)))) AS bit027
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -028)))) AS bit028
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -029)))) AS bit029
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -030)))) AS bit030
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -031)))) AS bit031
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -032)))) AS bit032
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -033)))) AS bit033
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -034)))) AS bit034
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -035)))) AS bit035
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -036)))) AS bit036
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -037)))) AS bit037
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -038)))) AS bit038
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -039)))) AS bit039
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -040)))) AS bit040
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -041)))) AS bit041
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -042)))) AS bit042
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -043)))) AS bit043
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -044)))) AS bit044
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -045)))) AS bit045
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -046)))) AS bit046
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -047)))) AS bit047
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -048)))) AS bit048
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -049)))) AS bit049
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -050)))) AS bit050
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -051)))) AS bit051
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -052)))) AS bit052
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -053)))) AS bit053
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -054)))) AS bit054
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -055)))) AS bit055
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -056)))) AS bit056
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -057)))) AS bit057
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -058)))) AS bit058
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -059)))) AS bit059
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -060)))) AS bit060
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -061)))) AS bit061
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -062)))) AS bit062
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -063)))) AS bit063
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -064)))) AS bit064
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -065)))) AS bit065
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -066)))) AS bit066
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -067)))) AS bit067
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -068)))) AS bit068
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -069)))) AS bit069
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -070)))) AS bit070
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -071)))) AS bit071
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -072)))) AS bit072
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -073)))) AS bit073
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -074)))) AS bit074
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -075)))) AS bit075
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -076)))) AS bit076
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -077)))) AS bit077
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -078)))) AS bit078
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -079)))) AS bit079
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -080)))) AS bit080
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -081)))) AS bit081
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -082)))) AS bit082
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -083)))) AS bit083
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -084)))) AS bit084
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -085)))) AS bit085
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -086)))) AS bit086
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -087)))) AS bit087
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -088)))) AS bit088
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -089)))) AS bit089
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -090)))) AS bit090
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -091)))) AS bit091
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -092)))) AS bit092
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -093)))) AS bit093
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -094)))) AS bit094
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -095)))) AS bit095
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -096)))) AS bit096
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -097)))) AS bit097
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -098)))) AS bit098
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -099)))) AS bit099
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -100)))) AS bit100
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -101)))) AS bit101
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -102)))) AS bit102
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -103)))) AS bit103
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -104)))) AS bit104
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -105)))) AS bit105
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -106)))) AS bit106
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -107)))) AS bit107
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -108)))) AS bit108
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -109)))) AS bit109
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -110)))) AS bit110
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -111)))) AS bit111
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -112)))) AS bit112
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -113)))) AS bit113
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -114)))) AS bit114
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -115)))) AS bit115
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -116)))) AS bit116
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -117)))) AS bit117
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -118)))) AS bit118
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -119)))) AS bit119
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -120)))) AS bit120
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -121)))) AS bit121
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -122)))) AS bit122
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -123)))) AS bit123
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -124)))) AS bit124
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -125)))) AS bit125
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -126)))) AS bit126
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -127)))) AS bit127
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -128)))) AS bit128
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -129)))) AS bit129
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -130)))) AS bit130
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -131)))) AS bit131
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -132)))) AS bit132
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -133)))) AS bit133
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -134)))) AS bit134
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -135)))) AS bit135
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -136)))) AS bit136
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -137)))) AS bit137
FROM PDCRINFO.DBQLOGTBL_HST as A
JOIN dim_user as U
  on a.UserName = u.UserName
WHERE LogDate BETWEEN date-7  and date-1
GROUP BY 1,2,3,4,5;


Success: 21 rows affected

Success: 21 rows affected

In [140]:
select  * from adlste_coa_stg.stg_dat_feature_usage_log

Site_ID,LogDate,User_Bucket,User_Department,DBSVersion,Request_Count,bit016,bit017,bit018,bit019,bit020,bit021,bit022,bit023,bit024,bit025,bit026,bit027,bit028,bit029,bit030,bit031,bit032,bit033,bit034,bit035,bit036,bit037,bit038,bit039,bit040,bit041,bit042,bit043,bit044,bit045,bit046,bit047,bit048,bit049,bit050,bit051,bit052,bit053,bit054,bit055,bit056,bit057,bit058,bit059,bit060,bit061,bit062,bit063,bit064,bit065,bit066,bit067,bit068,bit069,bit070,bit071,bit072,bit073,bit074,bit075,bit076,bit077,bit078,bit079,bit080,bit081,bit082,bit083,bit084,bit085,bit086,bit087,bit088,bit089,bit090,bit091,bit092,bit093,bit094,bit095,bit096,bit097,bit098,bit099,bit100,bit101,bit102,bit103,bit104,bit105,bit106,bit107,bit108,bit109,bit110,bit111,bit112,bit113,bit114,bit115,bit116,bit117,bit118,bit119,bit120,bit121,bit122,bit123,bit124,bit125,bit126,bit127,bit128,bit129,bit130,bit131,bit132,bit133,bit134,bit135,bit136,bit137
{siteid},2020-06-10,Admin,dept,16.20.32.1,292114,124,158591,157838,1691,158591,0,158591,944,0,99,1801,282,282,1541,0,0,0,0,172,21389,102,7,164411,323,164893,172,20,0,0,0,0,0,172,0,0,16,66,0,497,0,0,0,0,42,289398,289398,289398,0,0,289398,0,289398,289398,0,200,0,0,3138,0,0,0,99107,0,0,0,1326,979,870,158591,40921,0,404,35737,0,7440,155506,0,900,0,468,0,76,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,404,0,0,0,1411,0,1149,1422,0,0,0,0,0,0,0,3235,0,0,0,0,0,0,0,22,0
{siteid},2020-06-06,Admin,dept,16.20.32.1,238602,124,125373,124748,1110,125373,0,125373,934,0,94,1804,285,285,1541,0,0,0,0,144,10381,9,8,130896,266,131266,144,18,0,0,0,0,0,144,0,0,16,62,0,497,0,0,0,0,48,235999,235999,235999,0,0,235999,0,235999,235999,0,194,0,0,0,0,0,0,73908,0,0,0,0,196,810,125373,28461,0,206,24342,0,2903,123155,0,882,0,679,0,73,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1229,0,1149,1446,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0
{siteid},2020-06-08,Admin,dept,16.20.32.1,258466,124,131994,129937,3736,131994,0,131994,947,0,169,1806,287,287,1541,0,0,0,0,192,11757,29,8,133711,282,134201,192,24,0,0,0,0,0,192,0,0,16,72,0,497,0,0,0,0,48,255844,255844,255844,0,0,255844,0,255844,255844,0,200,0,0,0,0,0,0,77383,0,0,0,44,539,1197,131994,30159,0,1369,26178,0,6525,128089,0,900,0,679,0,85,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1390,0,1159,1466,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24,0
{siteid},2020-06-05,Unknown,Unknown,16.20.32.1,4910921,5308,3423433,3138975,158022,3423433,0,3423435,43817,7,20330,47200,13618,39979,7230,6,2,5,0,3499,1374641,1,188486,1572675,6809,3327744,8961,60,78106,0,0,11,0,3078,2,3429,408896,1761,0,21620,0,0,3,3,16798,4877992,4877992,4877992,0,0,4877992,0,4877992,4877992,0,1936,0,0,0,3,4,0,978490,0,0,0,56372,109053,111719,3423435,2078103,185,26076,1606255,0,690981,3007948,49909,94214,221224,48166,0,5075,0,218,0,370,0,791,0,0,0,0,0,0,0,0,0,0,79,0,0,53151,0,27163,150220,127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,709,0
{siteid},2020-06-06,Unknown,Unknown,16.20.32.1,4652806,4144,3360559,2960483,302773,3360559,0,3360559,33419,0,22223,43023,13655,36315,6714,0,0,0,0,3417,1301496,0,135359,1421100,6507,3174064,6813,65,80813,0,0,0,0,3160,0,3368,101279,1463,0,18133,0,0,3,3,22217,4627223,4627223,4627223,0,0,4627223,0,4627223,4627223,0,1610,0,0,0,3,4,0,928146,0,0,0,57047,108842,95975,3360559,1973725,175,17067,1495028,0,882469,2757149,46528,84419,178281,142805,0,4298,0,102,0,336,0,438,0,0,0,0,0,0,0,0,0,0,47,0,0,76138,0,18585,170519,123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,610,0
{siteid},2020-06-09,Admin,dept,16.20.32.1,285571,124,143438,142653,1592,143438,0,143438,946,0,103,1804,285,285,1541,0,0,0,0,192,24247,21,8,151910,278,152347,192,24,0,0,0,0,0,192,0,0,16,70,0,497,0,0,0,0,48,281352,281352,281352,0,0,281352,0,281352,281352,0,193,0,0,0,0,0,0,116279,0,0,0,288,4161,859,143438,33943,0,475,31084,0,21277,136573,0,900,0,679,0,85,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1438,0,1157,1456,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,0
{siteid},2020-06-08,Unknown,Unknown,16.20.32.1,4985990,5254,3384572,2846264,322701,3384572,0,3384574,34108,247,96684,48052,14112,39496,8568,246,2,245,0,5018,1413382,17,121244,1501509,9224,3134582,723

<br><br><br><br>
# Test Process

In [2]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


###  Built Temp Table:   dim_users

In [3]:

CREATE MULTISET VOLATILE TABLE "dim_user.csv"
("SiteID"                      VARCHAR(111)  CHARACTER SET UNICODE 
,"Pattern_Type"                VARCHAR(105)  CHARACTER SET UNICODE 
,"Pattern"                     VARCHAR(116)  CHARACTER SET UNICODE 
,"User_Bucket"                 VARCHAR(110)  CHARACTER SET UNICODE 
,"User_Department"             VARCHAR(104)  CHARACTER SET UNICODE 
,"User_SubDepartment"          VARCHAR(107)  CHARACTER SET UNICODE 
,"User_Region"                 VARCHAR(103)  CHARACTER SET UNICODE 
,"Priority"                    BIGINT
) NO PRIMARY INDEX
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [4]:

INSERT INTO "dim_user.csv"
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('Viewpoint' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i0    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('TDWM' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i1    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('CRASHDUMPS' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i2    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('DBC' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i3    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Equal' as varchar(105))
 ,cast('DBMON' as varchar(116))
 ,cast('TDInternal' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(100 as bigint)
from (sel 1 one) i4    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%STG%USER%' as varchar(116))
 ,cast('ETL' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2100 as bigint)
from (sel 1 one) i5    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('BAR_%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2125 as bigint)
from (sel 1 one) i6    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_BAR' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2150 as bigint)
from (sel 1 one) i7    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('DSA_%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2175 as bigint)
from (sel 1 one) i8    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_DSA' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2200 as bigint)
from (sel 1 one) i9    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('ARC_%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2225 as bigint)
from (sel 1 one) i10    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_ARC' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2250 as bigint)
from (sel 1 one) i11    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('DBADMIN%' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2500 as bigint)
from (sel 1 one) i12    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_ADMIN' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2600 as bigint)
from (sel 1 one) i13    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_DBA' as varchar(116))
 ,cast('Admin' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(2800 as bigint)
from (sel 1 one) i14    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('CALLREP%' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(4300 as bigint)
from (sel 1 one) i15    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%Tableau%' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(4400 as bigint)
from (sel 1 one) i16    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%BOBJ%' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(4500 as bigint)
from (sel 1 one) i17    UNION ALL
SELECT
  cast('Default' as varchar(111))
 ,cast('Like' as varchar(105))
 ,cast('%_ETL_%' as varchar(116))
 ,cast('ETL' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(6000 as bigint)
from (sel 1 one) i18    UNION ALL
SELECT
  cast('TDCLOUD%TD%' as varchar(111))
 ,cast('RegEx' as varchar(105))
 ,cast('[a-z]{2}[0-9]{6}' as varchar(116))
 ,cast('Analytic' as varchar(110))
 ,cast('dept' as varchar(104))
 ,cast('subdept' as varchar(107))
 ,cast('rgn' as varchar(103))
 ,cast(7000 as bigint)
from (sel 1 one) i19    ;

Success: 20 rows affected

In [5]:

create volatile table dim_user as
(
  select
   '{siteid}' as Site_ID
  ,o.UserName
  ,o.UserHash
  ,coalesce(p.User_Bucket,'Unknown') as User_Bucket
  ,coalesce(p.User_Department, 'Unknown') as User_Department
  ,coalesce(p.User_SubDepartment, 'Unknown') as User_SubDepartment
  ,coalesce(p.User_Region, 'Unknown') as User_Region
  ,coalesce(p.Priority,1e6) as Priority_
  ,coalesce(p.Pattern_Type,'Equal')  as Pattern_Type
  ,coalesce(p.Pattern, o.UserName) as Pattern
  ,coalesce(p.SiteID, 'None')        as SiteID_
  from (select
         trim(DatabaseName) as UserName
        ,substr(Username,1,3) as first3
        ,substr(Username,floor(character_length(Username)/2)-1,3) as middle3
        ,substr(Username,character_length(Username)-3,3) as last3
        /* generate UserHash value */
        ,trim(cast(from_bytes(hashrow( Username),'base16') as char(9))) ||
         trim(cast(from_bytes(hashrow( first3  ),'base16') as char(9))) ||
         trim(cast(from_bytes(hashrow( middle3 ),'base16') as char(9))) ||
         trim(cast(from_bytes(hashrow( last3   ),'base16') as char(9))) as UserHash
        from dbc.DatabasesV where DBKind = 'U'
        ) as o
  left join "dim_user.csv" as p
    on (case
        when p.Pattern_Type = 'Equal' and o.UserName = p.Pattern then 1
        when p.Pattern_Type = 'Like'  and o.UserName like p.Pattern then 1
        when p.Pattern_Type = 'RegEx'
         and character_length(regexp_substr(o.UserName, p.Pattern,1,1,'i'))>0 then 1
        else 0 end) = 1
    and SiteID_ in('default','None') or '{siteid}' like SiteID_
  qualify Priority_ = min(Priority_)over(partition by o.UserName)
) with data
primary index (UserName)
on commit preserve rows
;


Success: 0 rows affected

In [6]:
drop table "dim_user.csv"
;

Success: 0 rows affected

In [7]:
collect stats on dim_user column(UserName)
;

Success: 0 rows affected

In [8]:
/*{{save:all_users.csv}}*/
Select top 10
 UserName, UserHash, User_Bucket
,User_Department, User_SubDepartment, User_Region
from dim_user
;

UserName,UserHash,User_Bucket,User_Department,User_SubDepartment,User_Region
sr122317,8C8B809-1E8DFD02-160611713A7079E4,Unknown,Unknown,Unknown,Unknown
RJS,14481C8514481C857B3706027B370602,Unknown,Unknown,Unknown,Unknown
KK250108,1B1D1CC4-7994BF48ADCA16B-45B29228,Unknown,Unknown,Unknown,Unknown
ZR186002,24A01A003EF09E86972E40B202CD97E,Unknown,Unknown,Unknown,Unknown
ND186026,2E6C25925954C733972E40B5135C088,Unknown,Unknown,Unknown,Unknown
ya250015,327EF82B-4C65D32DADCA16B51A8885B,Unknown,Unknown,Unknown,Unknown
as186104,28E1B844-2A1414F6972E40B1D95284,Unknown,Unknown,Unknown,Unknown
tb185024,E5F12FE-2F0895F157BD8F52-2758E714,Unknown,Unknown,Unknown,Unknown
zl255010,48D24DB-5EAFEE3B34427A6E20A1C56B,Unknown,Unknown,Unknown,Unknown
kk186073,31EC2A3-319A12CA972E40B1627B0E4,Unknown,Unknown,Unknown,Unknown


In [17]:
-- shortcut for testing purposes
Insert into adlste_coa_stg.stg_dat_feature_usage_log
SELECT
 'TDCLOUD14TD03' (VARCHAR(100)) as SiteID
,A.LogDate as LogDate
,u.User_Bucket
,u.User_Department
/*  dbsversion is required */
,(Select trim(infoData) as DBSVersion from dbc.dbcinfo where InfoKey = 'VERSION') AS DBSVersion
,count(*) as Request_Count
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -016)))) AS bit016
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -017)))) AS bit017
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -018)))) AS bit018
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -019)))) AS bit019
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -020)))) AS bit020
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -021)))) AS bit021
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -022)))) AS bit022
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -023)))) AS bit023
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -024)))) AS bit024
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -025)))) AS bit025
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -026)))) AS bit026
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -027)))) AS bit027
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -028)))) AS bit028
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -029)))) AS bit029
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -030)))) AS bit030
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -031)))) AS bit031
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -032)))) AS bit032
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -033)))) AS bit033
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -034)))) AS bit034
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -035)))) AS bit035
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -036)))) AS bit036
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -037)))) AS bit037
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -038)))) AS bit038
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -039)))) AS bit039
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -040)))) AS bit040
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -041)))) AS bit041
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -042)))) AS bit042
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -043)))) AS bit043
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -044)))) AS bit044
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -045)))) AS bit045
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -046)))) AS bit046
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -047)))) AS bit047
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -048)))) AS bit048
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -049)))) AS bit049
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -050)))) AS bit050
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -051)))) AS bit051
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -052)))) AS bit052
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -053)))) AS bit053
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -054)))) AS bit054
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -055)))) AS bit055
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -056)))) AS bit056
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -057)))) AS bit057
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -058)))) AS bit058
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -059)))) AS bit059
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -060)))) AS bit060
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -061)))) AS bit061
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -062)))) AS bit062
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -063)))) AS bit063
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -064)))) AS bit064
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -065)))) AS bit065
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -066)))) AS bit066
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -067)))) AS bit067
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -068)))) AS bit068
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -069)))) AS bit069
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -070)))) AS bit070
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -071)))) AS bit071
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -072)))) AS bit072
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -073)))) AS bit073
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -074)))) AS bit074
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -075)))) AS bit075
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -076)))) AS bit076
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -077)))) AS bit077
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -078)))) AS bit078
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -079)))) AS bit079
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -080)))) AS bit080
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -081)))) AS bit081
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -082)))) AS bit082
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -083)))) AS bit083
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -084)))) AS bit084
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -085)))) AS bit085
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -086)))) AS bit086
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -087)))) AS bit087
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -088)))) AS bit088
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -089)))) AS bit089
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -090)))) AS bit090
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -091)))) AS bit091
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -092)))) AS bit092
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -093)))) AS bit093
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -094)))) AS bit094
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -095)))) AS bit095
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -096)))) AS bit096
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -097)))) AS bit097
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -098)))) AS bit098
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -099)))) AS bit099
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -100)))) AS bit100
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -101)))) AS bit101
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -102)))) AS bit102
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -103)))) AS bit103
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -104)))) AS bit104
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -105)))) AS bit105
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -106)))) AS bit106
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -107)))) AS bit107
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -108)))) AS bit108
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -109)))) AS bit109
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -110)))) AS bit110
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -111)))) AS bit111
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -112)))) AS bit112
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -113)))) AS bit113
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -114)))) AS bit114
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -115)))) AS bit115
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -116)))) AS bit116
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -117)))) AS bit117
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -118)))) AS bit118
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -119)))) AS bit119
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -120)))) AS bit120
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -121)))) AS bit121
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -122)))) AS bit122
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -123)))) AS bit123
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -124)))) AS bit124
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -125)))) AS bit125
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -126)))) AS bit126
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -127)))) AS bit127
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -128)))) AS bit128
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -129)))) AS bit129
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -130)))) AS bit130
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -131)))) AS bit131
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -132)))) AS bit132
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -133)))) AS bit133
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -134)))) AS bit134
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -135)))) AS bit135
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -136)))) AS bit136
,ZEROIFNULL(SUM(GETBIT(A.FEATUREUSAGE,(2047 -137)))) AS bit137
FROM PDCRINFO.DBQLOGTBL_HST as A 
JOIN dim_user as U
  on a.UserName = u.UserName
WHERE LogDate BETWEEN DATE-8 and DATE-1
GROUP BY 1,2,3,4,5;

Success: 24 rows affected

In [64]:
drop table feature_usage_unpivot

Success: 0 rows affected

In [20]:
   /* unpivot data from source */
    create volatile table feature_usage_unpivot as
    (
        select *
        from adlste_coa_stg.stg_dat_feature_usage_log
        unpivot(Usage_Cnt for BitPos in
            (bit016 as '016' ,bit017 as '017' ,bit018 as '018' ,bit019 as '019' 
            ,bit020 as '020' ,bit021 as '021' ,bit022 as '022' ,bit023 as '023' ,bit024 as '024'
            ,bit025 as '025' ,bit026 as '026' ,bit027 as '027' ,bit028 as '028' ,bit029 as '029'
            ,bit030 as '030' ,bit031 as '031' ,bit032 as '032' ,bit033 as '033' ,bit034 as '034'
            ,bit035 as '035' ,bit036 as '036' ,bit037 as '037' ,bit038 as '038' ,bit039 as '039'
            ,bit040 as '040' ,bit041 as '041' ,bit042 as '042' ,bit043 as '043' ,bit044 as '044'
            ,bit045 as '045' ,bit046 as '046' ,bit047 as '047' ,bit048 as '048' ,bit049 as '049'
            ,bit050 as '050' ,bit051 as '051' ,bit052 as '052' ,bit053 as '053' ,bit054 as '054'
            ,bit055 as '055' ,bit056 as '056' ,bit057 as '057' ,bit058 as '058' ,bit059 as '059'
            ,bit060 as '060' ,bit061 as '061' ,bit062 as '062' ,bit063 as '063' ,bit064 as '064'
            ,bit065 as '065' ,bit066 as '066' ,bit067 as '067' ,bit068 as '068' ,bit069 as '069'
            ,bit070 as '070' ,bit071 as '071' ,bit072 as '072' ,bit073 as '073' ,bit074 as '074'
            ,bit075 as '075' ,bit076 as '076' ,bit077 as '077' ,bit078 as '078' ,bit079 as '079'
            ,bit080 as '080' ,bit081 as '081' ,bit082 as '082' ,bit083 as '083' ,bit084 as '084'
            ,bit085 as '085' ,bit086 as '086' ,bit087 as '087' ,bit088 as '088' ,bit089 as '089'
            ,bit090 as '090' ,bit091 as '091' ,bit092 as '092' ,bit093 as '093' ,bit094 as '094'
            ,bit095 as '095' ,bit096 as '096' ,bit097 as '097' ,bit098 as '098' ,bit099 as '099'
            ,bit100 as '100' ,bit101 as '101' ,bit102 as '102' ,bit103 as '103' ,bit104 as '104'
            ,bit105 as '105' ,bit106 as '106' ,bit107 as '107' ,bit108 as '108' ,bit109 as '109'
            ,bit110 as '110' ,bit111 as '111' ,bit112 as '112' ,bit113 as '113' ,bit114 as '114'
            ,bit115 as '115' ,bit116 as '116' ,bit117 as '117' ,bit118 as '118' ,bit119 as '119'
            ,bit120 as '120' ,bit121 as '121' ,bit122 as '122' ,bit123 as '123' ,bit124 as '124'
            ,bit125 as '125' ,bit126 as '126' ,bit127 as '127' ,bit128 as '128' ,bit129 as '129'
            ,bit130 as '130' ,bit131 as '131' ,bit132 as '132' ,bit133 as '133' ,bit134 as '134'
            ,bit135 as '135' ,bit136 as '136' ,bit137 as '137' ) 
        ) tmp
    ) with data 
      no primary index
      on commit preserve rows;

Success: 0 rows affected

In [60]:
update feature_usage_unpivot set DBSVersion = '16.20.49.01'
--update feature_usage_unpivot set DBSVersion = '16.20.33.00'

Success: 2928 rows affected

In [29]:
select * from feature_usage_unpivot
where DBSVersion between '16.20.33.01' and '16.20.53.07'

In [33]:
select top 10 * from adlste_coa.coat_dim_feature

Feature_Id,Product_Name,Feature_Name,DBS_Version,Feature_BitPos,Feature_SubCategory,Feature_Category,Product_Category,Objective_Name,Solution_Type,Business_Value_Desc,User_Experience_Level_Name,Initiator_Role_Category,Enabled_Thru,DBQL_FeatureInfo_Flag,Feature_DataSource,MBO_Ind,Differentiator_Ind,Feature_Weight,Feature_Notes,Feature_SQL_URL,Playbook_URL,Run_Id
12000,Service Catalysts,Consumption Analytics,DEFAULT,0,Data,Accelerator,Consulting,Get help through services,Accelerator,"Reduced time to insights, leverages Teradata expertise & best practices",,TBD,TBD,N,TBD,0,1,0,,TBD,TBD,0
15000,Vantage Analyst,Text,DEFAULT,0,Analyst UI,User Interface,Vantage,Visualize Analytics,Self-Service,Scalable Advanced Analytics,,TBD,TBD,N,TBD,1,1,5,,TBD,TBD,0
24000,Vantage Database,Analytics Extensions,DEFAULT,100,In-Database Programming,SQL Engine,Vantage,Run SQL,In-database Analytics,Scalable Advanced Analytics,,TBD,TBD,Y,TBD,-9,-9,0,,TBD,TBD,0
19900,Vantage Database,XML Data Type,DEFAULT,42,Advanced Data Types,SQL Engine,Vantage,Load Data,In-database Analytics,Exploit new Data Types,,TBD,TBD,Y,TBD,1,1,5,,TBD,TBD,0
21800,Vantage Database,Statement Info Parcel,DEFAULT,40,Connectivity - ODBC,SQL Engine,Vantage,Expose data to applications,Connectivity,Enhanced performance,,TBD,TBD,Y,TBD,0,0,0,,TBD,TBD,0
27900,Vantage Database,External Stored Procedure,DEFAULT,53,Programming Support,SQL Engine,Vantage,Run SQL,In-database Analytics,Scalable Advanced Analytics,,TBD,TBD,Y,TBD,1,1,5,,TBD,TBD,0
27700,Vantage Database,Index Analysis,DEFAULT,78,Performance Management,SQL Engine,Vantage,Manage my environment,Administration,Enhanced performance,,TBD,TBD,Y,TBD,-9,-9,0,,TBD,TBD,0
23200,Vantage Database,Geospatial Index Single Table Distance Predicate,DEFAULT,123,Geospatial,SQL Engine,Vantage,Run SQL,In-database Analytics,Scalable Advanced Analytics,,TBD,TBD,Y,TBD,1,1,2,,TBD,TBD,0
22100,Vantage Database,Not System-Default Map,DEFAULT,120,Data Distribution,SQL Engine,Vantage,Load Data,Performance,Simplified data management,,TBD,TBD,Y,TBD,0,1,0,,TBD,TBD,0
29500,Vantage Database,DBQL OBJECT USECOUNT Logging,DEFAULT,67,Usage Analysis,SQL Engine,Vantage,Understand my usage,Administration,Provides workload insights by user and data domain,,TBD,TBD,Y,TBD,0,1,0,,TBD,TBD,0


In [61]:

    INSERT into adlste_coa.coat_dat_feature_usage_log 
    select u.Site_ID, u.LogDate, f.Feature_ID
    ,case when u.DBSVersion between '16.20.33.01' and '16.20.53.07' then 'ALT01'
          else 'DEFAULT' end as DBS_Version
    ,u.User_Bucket, u.User_Department, 'na' as User_SubDepartment, 'na' as User_Region
    ,Usage_Cnt as Feature_Usage_Cnt
    ,0 as RunID
    from feature_usage_unpivot u
    join adlste_coa.coat_dim_feature f
      on cast(u.BitPos as int) = f.Feature_BitPos
     and f.DBS_Version = case 
                         when u.DBSVersion between '16.20.33.01' and '16.20.53.07' then 'ALT01'
                         else 'DEFAULT' end

In [42]:
select * from dbc.tablesv 
where tablename like '%feature%'
and databasename not like 'ADLSTE%'
and databasename = 'DBC'

DataBaseName,TableName,Version,TableKind,ProtectionType,JournalFlag,CreatorName,RequestText,CommentString,ParentCount,ChildCount,NamedTblCheckCount,UnnamedTblCheckExist,PrimaryKeyIndexId,RepStatus,CreateTimeStamp,LastAlterName,LastAlterTimeStamp,RequestTxtOverflow,AccessCount,LastAccessTimeStamp,UtilVersion,QueueFlag,CommitOpt,TransLog,CheckOpt,TemporalProperty,ResolvedCurrent_Date,ResolvedCurrent_Timestamp,SystemDefinedJI,VTQualifier,TTQualifier,PIColumnCount,PartitioningLevels,LoadProperty,CurrentLoadId,LoadIdLayout,DelayedJI,LastArchiveId,LastFullArchiveId,BlockSize,FreeSpacePercent,MergeBlockRatio,CheckSum,BlockCompression,BlockCompressionAlgorithm,BlockCompressionLevel,TableHeaderFormat,RowSizeFormat,MapName,ColocationName,TVMFlavor,FastAlterTable,IncrementalRestoreEnabled
DBC,QryLogFeatureUseJSON,1,V,F,NN,DBC,"REPLACE VIEW DBC.QryLogFeatureUseJSON AS LOCKING TABLE DBC.DBQLogTbl FOR ACCESS select JsonTbl.QueryID (FORMAT '--Z(17)9'), CAST(JsonTbl.FeatureUsageJSON a",Returns Feature usage information in JSON format.,0,0,0,N,,,2019-07-21 12:52:45,DBC,2019-07-21 12:52:45,,,,,N,N,Y,N,N,,,N,,,0,0,,,,,,,,,,,,,,,0,,,,,
DBC,QryLogFeatureListV,1,V,F,NN,DBC,"REPLACE VIEW DBC.QryLogFeatureListV AS SELECT FeatureBitPos, FeatureName FROM TABLE (SYSLIB.FeatureNames_TBF()) as t1;",Static table function view to list all the features being tracked and logged by Feature Use Logging.,0,0,0,N,,,2019-07-21 12:47:42,DBC,2019-07-21 12:47:42,,,,,N,N,Y,N,N,,,N,,,0,0,,,,,,,,,,,,,,,0,,,,,
DBC,QRYLOGFEATUREUSECOUNTV,1,V,F,NN,DBC,"REPLACE VIEW DBC.QRYLOGFEATUREUSECOUNTV AS SELECT CAST(B.FEATURENAME AS CHAR(100)) AS FEATURENAME, SUM(GETBIT(A.FEATUREUSAGE,(2047 - B.FEATUREBITP",The DBC.QRYLOGFEATUREUSECOUNTV view provides feature names and usage count.,0,0,0,N,,,2019-07-21 12:53:45,DBC,2019-07-21 12:53:45,,,,,N,N,Y,N,N,,,N,,,0,0,,,,,,,,,,,,,,,0,,,,,
